In [117]:
### import libs
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding,  Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [118]:
TRAINING_SIZE = 20000
EMBEDDING_DIM = 50
VOCAB_SIZE = 12000
NUM_EPOCHS = 30
MAX_LEN = 100

In [119]:
data = pd.read_csv("output/formated_reviews.csv")

In [120]:
reviwes = data["review"]
labels = data["score"]




In [121]:
traning_data = reviwes[0:TRAINING_SIZE]
traning_labels = labels[0:TRAINING_SIZE]

testing_data = reviwes[TRAINING_SIZE:]
testing_labels = labels[TRAINING_SIZE:]


In [130]:
training_df = pd.DataFrame({"review" : traning_data, "score" : traning_labels})

training_df.head()

training_df['score'].value_counts(normalize=True) * 100

score
0    73.395
1    26.605
Name: proportion, dtype: float64

In [122]:
tokenizer  = Tokenizer(num_words=VOCAB_SIZE , oov_token="<OOV>")
tokenizer.fit_on_texts(traning_data)

training_sequences = tokenizer.texts_to_sequences(traning_data)
training_padded = pad_sequences(training_sequences, maxlen=MAX_LEN, padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(testing_data)
testing_padded = pad_sequences(testing_sequences, maxlen=MAX_LEN, padding="post", truncating="post")

word_index = tokenizer.word_index

print("Found %s unique tokens." % len(word_index))

Found 11193 unique tokens.


In [123]:
print(training_padded.shape[1])

100


In [124]:
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=training_padded.shape[1]),
    GlobalAveragePooling1D(),
    Dense(300, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [125]:
model.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [126]:
history = model.fit(training_padded, traning_labels, epochs=NUM_EPOCHS, validation_data=(testing_padded, testing_labels))

Epoch 1/30


625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7519 - loss: 0.5202 - val_accuracy: 0.8536 - val_loss: 0.3212
Epoch 2/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8604 - loss: 0.3183 - val_accuracy: 0.8743 - val_loss: 0.2867
Epoch 3/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8876 - loss: 0.2690 - val_accuracy: 0.8819 - val_loss: 0.2799
Epoch 4/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8981 - loss: 0.2489 - val_accuracy: 0.8801 - val_loss: 0.2804
Epoch 5/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8929 - loss: 0.2534 - val_accuracy: 0.8862 - val_loss: 0.2737
Epoch 6/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9062 - loss: 0.2293 - val_accuracy: 0.8467 - val_loss: 0.3388
Epoch 7/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9129 - loss: 0.2201 - val_accuracy: 0.8832 - val_loss: 0.2853
Epoch 8/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9241 - loss: 0.1866 - val_accuracy: 0.8932 - val_

In [134]:
### Testing 


sentence = ["Odiei esse produto! muito ruim", "o vendedor chutou meu cachorro"  , "eu amo lana del rey"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=MAX_LEN, padding="post", truncating="post")
print(model.predict(padded))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[[9.8483354e-01]
 [4.1189367e-01]
 [1.1541868e-02]
 [8.6677470e-04]
 [4.4850799e-01]]
